# <center>EC3382 - Assignment 3</center>

In [1]:
import numpy as np
import pandas as pd

In this assignment,  I want you to take the skills you have learned to define a trading strategy and get used to how it works. Then, I want you to optimize it on some price data. In the process, you will write a mini broker. This gets you thinking about what happens once your trading program interacts with Quantopian. This is clearly an abstraction but all the questions that you will ask doing this assignment are exactly the ones that you should be asking when you code on Quantopian. 

We downloaded some daily historical prices from Yahoo Finance. These data were downloaded via navigating to the URL:
http://ichart.finance.yahoo.com/table.csv?s=BAC&a=00&b=01&c=2000&d=11&e=31&f=2009&g=d&ignore=.csv
Where the letters BAC (CMA,USB,WFC) represent the ticker and the parameters (a-g) represent the from and to dates. loadData.py loads the downloaded data as a pandas frame. Take a look to see how this works. For the grading, I will use different tickers and different dates as the data.

In [2]:
from loadData import BAC, CMA, USB, WFC
#You should print these out to see what you are working with

In [3]:
BAC.head()

,Date,Open,High,Low,Close,Volume,Adj Close
2514,2000-01-03,50.2500,50.2500,48.0000,48.4375,13705800,15.289114
2513,2000-01-04,47.7500,47.9375,44.9375,45.5625,27293400,14.381631
2512,2000-01-05,45.0625,46.4375,44.5000,46.0625,22855600,14.539455
2511,2000-01-06,46.9375,50.0000,46.7500,50.0000,17307000,15.782312
2510,2000-01-07,49.6250,49.6250,48.0000,48.6875,11632800,15.368026


In [4]:
# CMA.head()

In [5]:
# USB.head()

In [6]:
# WFC.head()

## Part A: Combining the data into a Pandas dataframe

Write a function which takes as inputs:  
a) a list of Pandas dataframes formatted like the one given above  
b) a list of ticker names  

and returns a new dataframe containing the adjusted prices of the input dataframes and the date. Rename adjusted prices so that the column name is the ticker symbol. Use the df.merge command to join the dataframes together. http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

Use an inner join using 'Date' as a key. In doing this, dates where one of the stocks was not traded will be excluded. This is strictly not kosher for trading and you should be aware of it. In commercial trading systems, you will have to make sure that the algorithm only runs when all the data is "as expected". Code defensively. When something unexpected, your code needs to choke. For now, we ignore this issue.

```python 
allPrices = combineData([BAC, CMA, USB],['BAC','CMA','USB'])
print allPrices[0:3]
```

the output would be

```python 
         Date        BAC        CMA        USB
0  2000-01-03  15.289114  27.197519  10.278392
1  2000-01-04  14.381631  26.064289   9.931932
2  2000-01-05  14.539455  25.988740   9.903059
```

In [7]:
#Example answer. Replace with pass to release
def combineData(inputList,colName):
    for j in range(len(inputList)):
        inputList[j] = inputList[j][["Date","Adj Close"]]
    master_df = inputList[0]
    for i in range(len(inputList)-1):
        master_df = master_df.merge(inputList[i+1], on="Date")
    newColNames = ["Date"]
    for k in range(len(colName)):
        newColNames.append(colName[k])
    master_df.columns = newColNames
    return master_df

allPrices = combineData([BAC, CMA, USB],['BAC','CMA','USB'])
print allPrices[0:3]

         Date        BAC        CMA        USB
0  2000-01-03  15.289114  27.197519  10.278392
1  2000-01-04  14.381631  26.064289   9.931932
2  2000-01-05  14.539455  25.988740   9.903059


## Part B: Mean reversion strategy

Create a function that takes as input  
a) a data frame formatted like BAC (Columsn: Date, BAC) given above. This is a subset of allPrices that you have just constructed.  
b) a parameter for how many lag periods to use in calculating moving average  
c) a parameter for the multiples of standard deviation as a threshold  

and returns a dataframe with an additional column called Order. Order is -1 if the current price strictly exceeds its k day moving average plus m times the k day moving standard deviation and returns 1 if the price is strictly lower than the k day moving average minus m times the k day moving standard deviation. 

Remember that a k day moving average for the mean and standard deviation will require k datapoints. Hence, your function should return 0 whenever there is not enough data to evaluate the strategy. 

Assume that the first column is always Date, and the second column is always some price of a ticker symbol (not necc. BAC).

If the input is...
```python
print allPrices.ix[0:10,0:2]
          Date        BAC
0   2000-01-03  15.289114
1   2000-01-04  14.381631
2   2000-01-05  14.539455
3   2000-01-06  15.782312
4   2000-01-07  15.368026
5   2000-01-10  14.835373
6   2000-01-11  14.519727
7   2000-01-12  14.677550
8   2000-01-13  15.012924
9   2000-01-14  15.940135
10  2000-01-18  15.151019     


dfTrades = meanRevert(df,k = 3,m = 0.1)
print dfTrades
          Date        BAC  Order
          Date        BAC  Order
0   2000-01-03  15.289114      0
1   2000-01-04  14.381631      0
2   2000-01-05  14.539455      1
3   2000-01-06  15.782312     -1
4   2000-01-07  15.368026     -1
5   2000-01-10  14.835373      1
6   2000-01-11  14.519727      1
7   2000-01-12  14.677550      0
8   2000-01-13  15.012924     -1
9   2000-01-14  15.940135     -1
10  2000-01-18  15.151019      1
```



In [8]:
def meanRevert(df,k,m):
    df["Roll_mean"] = pd.DataFrame(df.iloc[:,1]).rolling(window=k,center=False).mean()
    df["Roll_sd"] = pd.DataFrame(df.iloc[:,1]).rolling(window=k,center=False).std()
    df["Comparison_top"] = df.Roll_mean + m*df.Roll_sd
    df["Comparison_bottom"] = df.Roll_mean - m*df.Roll_sd
    df.fillna(0, inplace=True)
    df["Order"] = np.where(df[df.columns[1]] > df.Comparison_top,-1,0)
    df["Order"] = np.where((df[df.columns[1]] < df.Comparison_bottom), 1, df["Order"])
    df = df.drop(["Roll_mean", "Roll_sd","Comparison_top","Comparison_bottom"], axis=1)
    df.loc[0:(k-2),"Order"] = 0
    return df

dfTrades = meanRevert(df = allPrices.iloc[0:11,0:2],k = 3, m = 0.1)
dfTrades

,Date,BAC,Order
0,2000-01-03,15.289114,0
1,2000-01-04,14.381631,0
2,2000-01-05,14.539455,1
3,2000-01-06,15.782312,-1
4,2000-01-07,15.368026,-1
5,2000-01-10,14.835373,1
6,2000-01-11,14.519727,1
7,2000-01-12,14.677550,0
8,2000-01-13,15.012924,-1
9,2000-01-14,15.940135,-1


In [9]:
def get_mean(a):
    no_nans_a = [value for value in a if not np.isnan(value)]
    return np.sum(no_nans_a) / len(no_nans_a)

def get_stddev(a):
    m = get_mean(a)
    no_nans_a = [value for value in a if not np.isnan(value)]
    sumy = 0
    for item in no_nans_a:
        sumy += (item - m)**2
    return (sumy/(len(no_nans_a)-1))**.5

def get_ratio(a, baseline):
    returns = get_returns(a)
    retdiff = returns - baseline
    return get_mean(retdiff)/get_stddev(retdiff)

def get_returns(a):
    percentages = [np.nan]
    for i in range(len(a)-1):
        percentages.append((a[i+1]-a[i])/a[i])
    return np.around(percentages, decimals=8)

def get_excess_returns(returns, baseline):
    return returns - baseline


## Part C: Trading

Construct a simple function (robotBroker) that takes  
a) a dataframe formatted like dfTrades  
b) Money. eg: https://www.youtube.com/watch?v=l91ISfcuzDw   

and outputs  
a) the sharpe ratio

Your function will simply keep track of the value of your portfolio (including money starting from the initial amount). Finally, it computes the returns and the unadjusted Sharpe Ratio from the Value column.

To illustrate, I made the prices easy to work with. The initial cash in this example is 10
```python 
          Date  BAC  Order Position Value
1   2009-12-28  1      0      0     10  = starting amount
2   2009-12-29  5      1      1     10  = 5 cash + 5 long
3   2009-12-30  4     -2      -1     9  = 13 cash - 4 cover
```

You *may* want to follow the data structure illustrated here. I am only interested in the Sharpe Ratio at the end. While there is a lot of code to write, a certain concept called copy-paste will come in useful! Take a look at Assignment 2. 

```python
A more realistic intermediate step. I only care about the Sharpe ratio
          Date        BAC  Order      Value
0   2000-01-03  15.289114      0  10.000000
1   2000-01-04  14.381631      0  10.000000
2   2000-01-05  14.539455      1  10.000000
3   2000-01-06  15.782312     -1  11.242857
4   2000-01-07  15.368026     -1  11.242857
5   2000-01-10  14.835373      1  11.775510
6   2000-01-11  14.519727      1  11.775510
7   2000-01-12  14.677550      0  11.933333
8   2000-01-13  15.012924     -1  12.268707
9   2000-01-14  15.940135     -1  12.268707
10  2000-01-18  15.151019      1  13.057823

sharpe = robotBroker(dfTrades,initCash = 10)
print sharpe
0.67749700464
```


In [10]:
def robotBroker(df, initCash):
    df["Value"] = initCash
    stock = df.columns[1]
    df["Stock_val"] = 0
    df["Position"] = 0
    for j in range(len(df.Order)):
        df.loc[j,"Position"] = np.sum(df.loc[0:j,"Order"])
        df.loc[j, "Stock_val"] = df.loc[j, stock] * df.loc[j, "Position"]
    df["Cash"] = initCash
    for i in range(1, len(df.Order)):
        prev_cash = df.loc[i-1,"Cash"]
        cash = prev_cash - (df.loc[i, "Order"]*df.loc[i, stock])
        df.loc[i, "Cash"] = cash
    df["Value"] = df.Cash + df.Stock_val
    riskfree = np.zeros(df.Value.count())
    return get_ratio(df.Value, riskfree)


sharpe = robotBroker(dfTrades,initCash = 10)
sharpe

0.67749701409101648

## Part D: Optimization
Let's put it all together! We want to try out different strategy parameters on different stocks. I want you to think about why you are finding what you are finding as you play with your strategy. Think BIG. Think about what is possible. This is just a simple motivational example.

Construct a function that takes as inputs:

a) A dataframe formatted like "allPrices"  
b) A list of tickers contained within allPrice, ie, ['BAC','USB']. These are the tickers to test.  
c) A list of lags k to try out, eg [3,4,5]
d) A list of cutoffs m to try out, eg [0.1,0.2,0.5]
e) An initial sum of money eg 100


The output:  
i) kBest a list containing the best choice of kBest for each of the input tickers in (b)  
ii) mBest a list containing the best choice of m for each of the input tickers in (b)  
iii) The corresponding sharpe ratio

Example
```python
[kBest, mBest, maxSharpe] = optimizeParams(allPrices[0:10],['BAC','USB'],[3,4,5],[0.1,0.2,0.5],100)
print kBest, mBest, maxSharpe
[3, 3] [0.5, 0.1] [0.75110882013827451, 0.51505589272506302]
```

In [42]:
def optimizeParams(df, tickerList, kList, mList, initCash):
    listty = []
    for stock in tickerList:
        bestSharpe = 0
        best_k = 0
        best_m = 0
        ticker_df = df.loc[:,["Date", stock]]
        for k in kList:
            for m in mList:
                working_df = meanRevert(ticker_df,k,m)
                current_sharpe = robotBroker(working_df, initCash)
                if current_sharpe > bestSharpe:
                    bestSharpe = current_sharpe
                    best_k = k
                    best_m = m
        listty.append([stock,best_k, best_m, bestSharpe])
#     print listty
    kBest = []
    mBest = []
    maxSharpe = []
    for lst in listty:
        kBest.append(lst[1])
        mBest.append(lst[2])
        maxSharpe.append(lst[3])
#     print zip(listty[0:2])
    return kBest,mBest,maxSharpe
                              
[kBest, mBest, maxSharpe] = optimizeParams(allPrices[0:10],['BAC','USB'],[3,4,5],[0.1,0.2,0.5],100)
print kBest, mBest, maxSharpe

[3, 3] [0.5, 0.1] [0.75110910296904509, 0.51505540101753122]
